In [32]:
"""
Блок 1. Оптимизация с помощью индекса
"""

import libs.db_connection

# Интересует конкретный пользователь
sql1 = "explain (analyze) select * from block11 where user_id = 5"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Bitmap Heap Scan on block11  (cost=5.23..402.26 rows=102 width=17) (actual time=0.047..0.358 rows=77 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Recheck Cond: (user_id = 5)')]),
 RealDictRow([('QUERY PLAN', '  Heap Blocks: exact=77')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Index Scan on block1_user  (cost=0.00..5.20 rows=102 width=0) (actual time=0.032..0.032 rows=77 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Index Cond: (user_id = 5)')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.348 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 0.424 ms')])]

In [3]:
# Интересуют события у которых категория < 10
sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..124706.36 rows=79287 width=17) (actual time=3.618..320.126 rows=90067 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on block11  (cost=0.00..115777.66 rows=33036 width=17) (actual time=3.402..285.003 rows=30022 loops=3)')]),
 RealDictRow([('QUERY PLAN', '        Filter: (category < 10)')]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 3303311')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.119 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 1.132 ms, Inlining 0.000 ms, Optimization 0.771 ms, Emission 8.848 ms, 

In [4]:
# Создаем индекс
sql1 = "create index block1_user on block11 (user_id)"

            
libs.db_connection.execute(sql1)

In [5]:
# Как стало
sql1 = "explain (analyze) select * from block11 where user_id = 5"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Bitmap Heap Scan on block11  (cost=5.23..402.26 rows=102 width=17) (actual time=0.067..2.025 rows=77 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Recheck Cond: (user_id = 5)')]),
 RealDictRow([('QUERY PLAN', '  Heap Blocks: exact=77')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Index Scan on block1_user  (cost=0.00..5.20 rows=102 width=0) (actual time=0.045..0.046 rows=77 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Index Cond: (user_id = 5)')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.453 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 2.102 ms')])]

In [6]:
# А что с категориями ?
sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..124707.13 rows=79288 width=17) (actual time=2.760..284.136 rows=90067 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on block11  (cost=0.00..115778.33 rows=33037 width=17) (actual time=2.954..254.674 rows=30022 loops=3)')]),
 RealDictRow([('QUERY PLAN', '        Filter: (category < 10)')]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 3303311')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.187 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 1.092 ms, Inlining 0.000 ms, Optimization 0.733 ms, Emission 7.665 ms, 

In [7]:
# создаем индекс для категорий

sql1 = "create index block1_category on block11 (category)"

            
libs.db_connection.execute(sql1)

In [8]:
# Как теперь работает запрос по категориям

sql1 = "explain (analyze) select * from block11 where category < 10"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Bitmap Heap Scan on block11  (cost=886.92..68938.24 rows=79288 width=17) (actual time=23.109..226.283 rows=90067 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Recheck Cond: (category < 10)')]),
 RealDictRow([('QUERY PLAN', '  Heap Blocks: exact=48332')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Index Scan on block1_category  (cost=0.00..867.09 rows=79288 width=0) (actual time=10.389..10.389 rows=90067 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Index Cond: (category < 10)')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 1.044 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 230.559 ms')])]

In [9]:
# Удаляем индекс для категории

sql1 = "drop index block1_category"
libs.db_connection.execute(sql1)

In [10]:
# Составной индекс (Multicolumn index)

sql1 = "explain (analyze) select * from block11 where category < 10 and type = 'click'"
            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..127571.60 rows=3766 width=17) (actual time=4.346..299.991 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on block11  (cost=0.00..126195.00 rows=1569 width=17) (actual time=3.143..274.260 rows=1504 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: ((category < 10) AND ((type)::text = 'click'::text))")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 3331830')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.353 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 0.900 ms, Inlining 0.00

In [11]:
# Создаем составной индекс для группы и типа события

sql1 = "create index block1_category_type on block11 (category, type)"
            
libs.db_connection.execute(sql1)

In [12]:
# Как теперь работает запрос с фильтром по 2 полям

sql1 = "explain (analyze) select * from block11 where category < 10 and type = 'click'"

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Bitmap Heap Scan on block11  (cost=1070.26..13128.88 rows=3766 width=17) (actual time=2.180..26.653 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               "  Recheck Cond: ((category < 10) AND ((type)::text = 'click'::text))")]),
 RealDictRow([('QUERY PLAN', '  Heap Blocks: exact=4357')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Index Scan on block1_category_type  (cost=0.00..1069.32 rows=3766 width=0) (actual time=1.520..1.521 rows=4511 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               "        Index Cond: ((category < 10) AND ((type)::text = 'click'::text))")]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.412 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 26.989 ms')])]

In [13]:
# Удаляем составной индекс

sql1 = "drop index block1_category_type"
libs.db_connection.execute(sql1)

In [14]:
# Плохая селективность (часть 1)

sql1 = "explain (analyze) select * from block11 where type = 'show'"
libs.db_connection.select_data_dict_with_commit(sql1, [])


[RealDictRow([('QUERY PLAN',
               'Seq Scan on block11  (cost=0.00..188695.00 rows=9525000 width=17) (actual time=3.048..1057.816 rows=9499633 loops=1)')]),
 RealDictRow([('QUERY PLAN', "  Filter: ((type)::text = 'show'::text)")]),
 RealDictRow([('QUERY PLAN', '  Rows Removed by Filter: 500367')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.443 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 2')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 0.312 ms, Inlining 0.000 ms, Optimization 0.345 ms, Emission 2.490 ms, Total 3.147 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 1307.105 ms')])]

In [15]:
# Плохая селективность (часть 2)
sql1 = "create index block1_type2 on block11 (type)"
libs.db_connection.execute(sql1)

In [16]:
# Плохая селективность (часть 1)

sql1 = "explain (analyze) select * from block11 where type = 'show'"
libs.db_connection.select_data_dict_with_commit(sql1, [])


[RealDictRow([('QUERY PLAN',
               'Seq Scan on block11  (cost=0.00..188695.00 rows=9525000 width=17) (actual time=3.875..1029.861 rows=9499633 loops=1)')]),
 RealDictRow([('QUERY PLAN', "  Filter: ((type)::text = 'show'::text)")]),
 RealDictRow([('QUERY PLAN', '  Rows Removed by Filter: 500367')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.614 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 2')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 0.377 ms, Inlining 0.000 ms, Optimization 0.353 ms, Emission 3.269 ms, Total 3.999 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 1274.308 ms')])]

In [17]:
# Плохая селективность (часть 3)

sql1 = "explain (analyze) select type, count(1) from block11 group by type"
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Finalize GroupAggregate  (cost=127195.06..127195.56 rows=2 width=13) (actual time=1176.815..1180.898 rows=2 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Group Key: type')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Gather Merge  (cost=127195.06..127195.52 rows=4 width=13) (actual time=1176.807..1180.890 rows=6 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '        Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Sort  (cost=126195.03..126195.04 rows=2 width=13) (actual time=1146.315..1146.316 rows=2 loops=3)')]),
 RealDictRow([('QUERY PLAN', '              Sort Key: type')]),
 RealDictRow([('QUERY PLAN',
               '              Sort Method: quicksort  Memory: 25kB')]),
 RealDictRow([('QUERY PLAN',
               '              Worker 0:  Sort Method: quicksort  Memory: 25kB')]),
 RealDictRow([('QUERY PLAN',
               '           

In [34]:
# Задание: Нам нужно загрузить таблицу из источника. 
# При этом забирать ее будем инкрементально каждый день за прошлые сутки 
# селект уже есть мы его попробовали, но он что-то тормозит, нужно посмотреть, можно ли его ускорить

sql = "explain (analyze) select * from task1 where dt = '2022-05-19 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])


[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1000.00..116779.03 rows=1 width=21) (actual time=511.786..514.605 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Seq Scan on task1  (cost=0.00..115778.93 rows=1 width=21) (actual time=479.000..479.000 rows=0 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               "        Filter: (dt = '2022-05-19 00:00:00+00'::timestamp with time zone)")]),
 RealDictRow([('QUERY PLAN', '        Rows Removed by Filter: 3333333')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.201 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')]),
 RealDictRow([('QUERY PLAN',
               '  Options: Inlining false, Optimization false, Expressions true, Deforming true')]),
 RealDictRow([('QUERY PLAN',
               '  Timing: Generation 2.669 ms, Inlining 0.000 ms,

In [35]:
sql_index = "create index task1_dt on task1 (dt)"
libs.db_connection.execute(sql_index)

In [41]:
# Как сейчас стало

sql = "explain (analyze) select * from task1 where dt = '2022-05-19 00:00:00'"
libs.db_connection.select_data_dict_with_commit(sql, [])

[RealDictRow([('QUERY PLAN',
               'Index Scan using task1_dt on task1  (cost=0.43..8.45 rows=1 width=21) (actual time=0.004..0.005 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               "  Index Cond: (dt = '2022-05-19 00:00:00+00'::timestamp with time zone)")]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.225 ms')]),
 RealDictRow([('QUERY PLAN', 'Execution Time: 0.033 ms')])]